In [1]:
import utils, RKHS_DAGMA_extractj
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
from CausalDisco.analytics import r2_sortability, var_sortability
from CausalDisco.baselines import r2_sort_regress, var_sort_regress
from cdt.metrics import SID, SHD
import seaborn as sns
import cdt
import multiprocessing
import threading
import json

Detecting 1 CUDA device(s).


In [2]:
sns.set_context("paper")

torch.set_default_dtype(torch.float64)

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)

utils.set_random_seed(0)
torch.manual_seed(0)

cdt.SETTINGS.rpath= 'C:\Program Files\R\R-4.3.3\\bin\Rscript'

num_cores = multiprocessing.cpu_count()

# Print the number of CPU cores
# print("Number of CPU cores:", num_cores)
# print(torch.__version__)

# Generate Data

In [78]:
n, d, s0, graph_type, sem_type = 100, 10, 40, 'ER', 'gp-add' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [79]:
n, d, s0, graph_type, sem_type = 100, 10, 40, 'ER', 'gp' # s0 = expected number of edges
B_true2 = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X2 = utils.simulate_nonlinear_sem(B_true2, n, sem_type) # [n, d] sample matrix
X2 = torch.from_numpy(X2)

In [80]:
n, d, s0, graph_type, sem_type = 100, 10, 40, 'ER', 'mlp' # s0 = expected number of edges
B_true3 = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X3 = utils.simulate_nonlinear_sem(B_true3, n, sem_type) # [n, d] sample matrix
X3 = torch.from_numpy(X3)

In [3]:
n, d, s0, graph_type, sem_type = 100, 10, 20, 'ER', 'mlp' # s0 = expected number of edges
B_true4 = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X4 = utils.simulate_nonlinear_sem(B_true4, n, sem_type) # [n, d] sample matrix
X4 = torch.from_numpy(X4)

In [91]:
n, d, s0, graph_type, sem_type = 100, 10, 20, 'ER', 'gp' # s0 = expected number of edges
B_true5 = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X5 = utils.simulate_nonlinear_sem(B_true5, n, sem_type) # [n, d] sample matrix
X5 = torch.from_numpy(X5)

# Do RKHS_DAGMA Discovery

## Find best hyperparameter 

In [7]:
def run_model(lambda1, tau, X, device, B_true):
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    results = {}
    
    eq_model = RKHS_DAGMA_extractj.DagmaRKHS(n, d, "gaussian").to(device)
    model = RKHS_DAGMA_extractj.DagmaRKHS_nonlinear(eq_model)
    X = X.to(device)
    start_mse = eq_model.mse(X).detach().cpu().numpy()
    time_start = time.time()
    W_est_no_thresh, output = model.fit(X, lambda1=lambda1, tau=tau, lr=0.03, w_threshold=0.0)
    time_end = time.time()

    thresh_values = [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
    for thresh in thresh_values:
        try:
            W_est_dagma = abs(W_est_no_thresh) * (abs(W_est_no_thresh) > thresh)
            acc_dagma = utils.count_accuracy(B_true, W_est_dagma != 0)
            valid = "yes"
        #sid_dagma = SID(B_true, W_est_dagma != 0).item()
        except Exception as e:
            W_est_dagma = results_var_sort_regress
            acc_dagma = utils.count_accuracy(B_true, W_est_dagma != 0)
            valid = "no"
            
        diff_dagma = np.linalg.norm(W_est_dagma - abs(B_true))
        mse_dagma = eq_model.mse(X).detach().cpu().numpy()
        h_val = eq_model.h_func(X, s=1).detach().cpu().numpy()
        W_est_dagma = W_est_dagma.tolist()
        B = B_true.tolist()
        key = f'lambda_{lambda1}_tau_{tau}_thresh_{thresh}'
        results[key] = {
            'SHD': acc_dagma['shd'],
            'TPR': acc_dagma['tpr'],
            'Time Elapsed': time_end - time_start,
            'F1': acc_dagma['f1'],
            'diff': diff_dagma,
            'mse': mse_dagma.item(),
            'valid': valid,
            'h_val': h_val,
            'start mse': start_mse
            #'W_est_dagma': W_est_dagma,
            #'B_true': B
        }

    torch.cuda.empty_cache()

    # filename = f'model_results_lambda_{lambda1}_tau_{tau}.txt'

    # with open(filename, 'w') as file:
    #     json.dump(results, file, indent=4) 

    return results

In [5]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print('device:', device)
torch.set_default_device(device)

device: cuda


In [46]:
results = run_model(lambda1 = 1e-3, tau = 1e-3, X = X, device = device, B_true = B_true)
print(results)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.001_thresh_0.005': {'SHD': 24, 'TPR': 0.425, 'Time Elapsed': 210.4406533241272, 'F1': 0.5666666666666667, 'diff': 6.4358218841446275, 'mse': 2.759302481270854, 'valid': 'no'}, 'lambda_0.001_tau_0.001_thresh_0.01': {'SHD': 24, 'TPR': 0.425, 'Time Elapsed': 210.4406533241272, 'F1': 0.5666666666666667, 'diff': 6.4358218841446275, 'mse': 2.759302481270854, 'valid': 'no'}, 'lambda_0.001_tau_0.001_thresh_0.05': {'SHD': 24, 'TPR': 0.45, 'Time Elapsed': 210.4406533241272, 'F1': 0.5538461538461539, 'diff': 5.9727437247036725, 'mse': 2.759302481270854, 'valid': 'yes'}, 'lambda_0.001_tau_0.001_thresh_0.1': {'SHD': 27, 'TPR': 0.35, 'Time Elapsed': 210.4406533241272, 'F1': 0.4999999999999999, 'diff': 6.017062395145129, 'mse': 2.759302481270854, 'valid': 'yes'}, 'lambda_0.001_tau_0.001_thresh_0.15': {'SHD': 36, 'TPR': 0.125, 'Time Elapsed': 210.4406533241272, 'F1': 0.21739130434782608, 'diff': 6.178023065643537, 'mse': 2.759302481270854, 'valid': 'yes'}, 'lambda_0.001_tau_0.001_

In [47]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true)
print(results)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005': {'SHD': 24, 'TPR': 0.425, 'Time Elapsed': 180.84269857406616, 'F1': 0.5666666666666667, 'diff': 6.4358218841446275, 'mse': 2.755426866017596, 'valid': 'no'}, 'lambda_0.001_tau_0.0001_thresh_0.01': {'SHD': 14, 'TPR': 0.775, 'Time Elapsed': 180.84269857406616, 'F1': 0.7380952380952381, 'diff': 5.834193380400272, 'mse': 2.755426866017596, 'valid': 'yes'}, 'lambda_0.001_tau_0.0001_thresh_0.05': {'SHD': 20, 'TPR': 0.6, 'Time Elapsed': 180.84269857406616, 'F1': 0.64, 'diff': 5.880015334999827, 'mse': 2.755426866017596, 'valid': 'yes'}, 'lambda_0.001_tau_0.0001_thresh_0.1': {'SHD': 26, 'TPR': 0.375, 'Time Elapsed': 180.84269857406616, 'F1': 0.5263157894736842, 'diff': 5.966769141733165, 'mse': 2.755426866017596, 'valid': 'yes'}, 'lambda_0.001_tau_0.0001_thresh_0.15': {'SHD': 34, 'TPR': 0.175, 'Time Elapsed': 180.84269857406616, 'F1': 0.29166666666666663, 'diff': 6.119084044361985, 'mse': 2.755426866017596, 'valid': 'yes'}, 'lambda_0.001_tau_0.0001_thre

In [86]:
results2 = run_model(lambda1 = 1e-3, tau = 1e-3, X = X2, device = device, B_true = B_true2)
print(results2)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.001_thresh_0.005': {'SHD': 35, 'TPR': 0.2, 'Time Elapsed': 181.4742386341095, 'F1': 0.26666666666666666, 'diff': 6.502116972713525, 'mse': 1.1107244185459668, 'valid': 'no', 'h_val': array(8.61615871e-09)}, 'lambda_0.001_tau_0.001_thresh_0.01': {'SHD': 35, 'TPR': 0.2, 'Time Elapsed': 181.4742386341095, 'F1': 0.26666666666666666, 'diff': 6.502116972713525, 'mse': 1.1107244185459668, 'valid': 'no', 'h_val': array(8.61615871e-09)}, 'lambda_0.001_tau_0.001_thresh_0.05': {'SHD': 18, 'TPR': 0.575, 'Time Elapsed': 181.4742386341095, 'F1': 0.6865671641791045, 'diff': 6.0667231735690965, 'mse': 1.1107244185459668, 'valid': 'yes', 'h_val': array(8.61615871e-09)}, 'lambda_0.001_tau_0.001_thresh_0.1': {'SHD': 36, 'TPR': 0.1, 'Time Elapsed': 181.4742386341095, 'F1': 0.18181818181818182, 'diff': 6.252252898571511, 'mse': 1.1107244185459668, 'valid': 'yes', 'h_val': array(8.61615871e-09)}, 'lambda_0.001_tau_0.001_thresh_0.15': {'SHD': 40, 'TPR': 0.0, 'Time Elapsed': 181.474238634

In [87]:
results2 = run_model(lambda1 = 1e-3, tau = 1e-4, X = X2, device = device, B_true = B_true2)
print(results2)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005': {'SHD': 35, 'TPR': 0.2, 'Time Elapsed': 163.8069553375244, 'F1': 0.26666666666666666, 'diff': 6.502116972713525, 'mse': 1.1084216716992488, 'valid': 'no', 'h_val': array(8.1489209e-09)}, 'lambda_0.001_tau_0.0001_thresh_0.01': {'SHD': 35, 'TPR': 0.2, 'Time Elapsed': 163.8069553375244, 'F1': 0.26666666666666666, 'diff': 6.502116972713525, 'mse': 1.1084216716992488, 'valid': 'no', 'h_val': array(8.1489209e-09)}, 'lambda_0.001_tau_0.0001_thresh_0.05': {'SHD': 14, 'TPR': 0.675, 'Time Elapsed': 163.8069553375244, 'F1': 0.75, 'diff': 5.973745884875571, 'mse': 1.1084216716992488, 'valid': 'yes', 'h_val': array(8.1489209e-09)}, 'lambda_0.001_tau_0.0001_thresh_0.1': {'SHD': 36, 'TPR': 0.1, 'Time Elapsed': 163.8069553375244, 'F1': 0.18181818181818182, 'diff': 6.240759172032432, 'mse': 1.1084216716992488, 'valid': 'yes', 'h_val': array(8.1489209e-09)}, 'lambda_0.001_tau_0.0001_thresh_0.15': {'SHD': 39, 'TPR': 0.025, 'Time Elapsed': 163.8069553375244, 'F1': 

In [88]:
results3 = run_model(lambda1 = 1e-3, tau = 1e-3, X = X3, device = device, B_true = B_true3)
print(results3)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.001_thresh_0.005': {'SHD': 24, 'TPR': 0.525, 'Time Elapsed': 319.3255410194397, 'F1': 0.49411764705882355, 'diff': 6.709012980221802, 'mse': 41.38616093191087, 'valid': 'yes', 'h_val': array(3.6323078e-07)}, 'lambda_0.001_tau_0.001_thresh_0.01': {'SHD': 24, 'TPR': 0.525, 'Time Elapsed': 319.3255410194397, 'F1': 0.49411764705882355, 'diff': 6.709012980221802, 'mse': 41.38616093191087, 'valid': 'yes', 'h_val': array(3.6323078e-07)}, 'lambda_0.001_tau_0.001_thresh_0.05': {'SHD': 24, 'TPR': 0.525, 'Time Elapsed': 319.3255410194397, 'F1': 0.49411764705882355, 'diff': 6.709012980221802, 'mse': 41.38616093191087, 'valid': 'yes', 'h_val': array(3.6323078e-07)}, 'lambda_0.001_tau_0.001_thresh_0.1': {'SHD': 24, 'TPR': 0.525, 'Time Elapsed': 319.3255410194397, 'F1': 0.49411764705882355, 'diff': 6.709012980221802, 'mse': 41.38616093191087, 'valid': 'yes', 'h_val': array(3.6323078e-07)}, 'lambda_0.001_tau_0.001_thresh_0.15': {'SHD': 24, 'TPR': 0.5, 'Time Elapsed': 319.325541019

In [89]:
results3 = run_model(lambda1 = 1e-3, tau = 1e-4, X = X3, device = device, B_true = B_true3)
print(results3)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005': {'SHD': 24, 'TPR': 0.525, 'Time Elapsed': 319.1279761791229, 'F1': 0.49411764705882355, 'diff': 6.7527533788326, 'mse': 41.35045746484701, 'valid': 'yes', 'h_val': array(4.03358675e-07)}, 'lambda_0.001_tau_0.0001_thresh_0.01': {'SHD': 24, 'TPR': 0.525, 'Time Elapsed': 319.1279761791229, 'F1': 0.49411764705882355, 'diff': 6.7527533788326, 'mse': 41.35045746484701, 'valid': 'yes', 'h_val': array(4.03358675e-07)}, 'lambda_0.001_tau_0.0001_thresh_0.05': {'SHD': 24, 'TPR': 0.525, 'Time Elapsed': 319.1279761791229, 'F1': 0.49411764705882355, 'diff': 6.7527533788326, 'mse': 41.35045746484701, 'valid': 'yes', 'h_val': array(4.03358675e-07)}, 'lambda_0.001_tau_0.0001_thresh_0.1': {'SHD': 24, 'TPR': 0.525, 'Time Elapsed': 319.1279761791229, 'F1': 0.49411764705882355, 'diff': 6.7527533788326, 'mse': 41.35045746484701, 'valid': 'yes', 'h_val': array(4.03358675e-07)}, 'lambda_0.001_tau_0.0001_thresh_0.15': {'SHD': 24, 'TPR': 0.5, 'Time Elapsed': 319.12797617

In [8]:
results4 = run_model(lambda1 = 1e-3, tau = 1e-4, X = X4, device = device, B_true = B_true4)
print(results4)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005': {'SHD': 28, 'TPR': 0.8, 'Time Elapsed': 182.41072154045105, 'F1': 0.5079365079365079, 'diff': 4.752326674047887, 'mse': 23.245976640421173, 'valid': 'yes', 'h_val': array(2.86495487e-07), 'start mse': array(215.07703297)}, 'lambda_0.001_tau_0.0001_thresh_0.01': {'SHD': 27, 'TPR': 0.8, 'Time Elapsed': 182.41072154045105, 'F1': 0.5161290322580645, 'diff': 4.752323601027349, 'mse': 23.245976640421173, 'valid': 'yes', 'h_val': array(2.86495487e-07), 'start mse': array(215.07703297)}, 'lambda_0.001_tau_0.0001_thresh_0.05': {'SHD': 27, 'TPR': 0.55, 'Time Elapsed': 182.41072154045105, 'F1': 0.42307692307692313, 'diff': 4.787476152994474, 'mse': 23.245976640421173, 'valid': 'yes', 'h_val': array(2.86495487e-07), 'start mse': array(215.07703297)}, 'lambda_0.001_tau_0.0001_thresh_0.1': {'SHD': 27, 'TPR': 0.4, 'Time Elapsed': 182.41072154045105, 'F1': 0.34782608695652173, 'diff': 4.832813844551872, 'mse': 23.245976640421173, 'valid': 'yes', 'h_val': array(

In [92]:
results5 = run_model(lambda1 = 1e-3, tau = 1e-4, X = X5, device = device, B_true = B_true5)
print(results5)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005': {'SHD': 27, 'TPR': 0.2, 'Time Elapsed': 194.50261282920837, 'F1': 0.19999999999999998, 'diff': 4.916404823149671, 'mse': 0.9406119373725568, 'valid': 'no', 'h_val': array(3.49188014e-08)}, 'lambda_0.001_tau_0.0001_thresh_0.01': {'SHD': 27, 'TPR': 0.2, 'Time Elapsed': 194.50261282920837, 'F1': 0.19999999999999998, 'diff': 4.916404823149671, 'mse': 0.9406119373725568, 'valid': 'no', 'h_val': array(3.49188014e-08)}, 'lambda_0.001_tau_0.0001_thresh_0.05': {'SHD': 27, 'TPR': 0.2, 'Time Elapsed': 194.50261282920837, 'F1': 0.19999999999999998, 'diff': 4.916404823149671, 'mse': 0.9406119373725568, 'valid': 'no', 'h_val': array(3.49188014e-08)}, 'lambda_0.001_tau_0.0001_thresh_0.1': {'SHD': 26, 'TPR': 0.3, 'Time Elapsed': 194.50261282920837, 'F1': 0.28571428571428564, 'diff': 4.311830584090073, 'mse': 0.9406119373725568, 'valid': 'yes', 'h_val': array(3.49188014e-08)}, 'lambda_0.001_tau_0.0001_thresh_0.15': {'SHD': 19, 'TPR': 0.15, 'Time Elapsed': 194.50

In [61]:
lambda1_values = [0, 1e-4, 1e-3, 1e-2, 1e-1]
tau_values = [0, 1e-4, 1e-3, 1e-2, 1e-1]
for lambda1 in lambda1_values:
    for tau in tau_values:
        run_model(lambda1, tau, X, device)
print("Gride search complete.")

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

...

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

Gride search complete.


In [32]:
run_model(lambda1 = 1e-4, tau = 0, X = X, device=device)

  0%|          | 0/23000.0 [00:00<?, ?it/s]

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [ ]:

lambda1_values = [0, 1e-4, 1e-3, 1e-2, 1e-1]
tau_values = [0, 1e-4, 1e-3, 1e-2, 1e-1]
#tau_values = [0]

threads = []
results = {}

# Create a thread for each combination of lambda1 and tau
for lambda1 in lambda1_values:
    for tau in tau_values:
        thread = threading.Thread(target=run_model, args=(lambda1, tau, X, device))
        threads.append(thread)
        thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

print("Gride search complete.")

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

...

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

  0%|          | 0/23000.0 [00:00<?, ?it/s]

In [ ]:
lambda1_values = [0, 1e-4, 1e-3, 1e-2, 1e-1]
tau_values = [0, 1e-4, 1e-3, 1e-2, 1e-1]
processes = []
for lambda1 in lambda1_values:
    for tau in tau_values:
        # Create a Process
        p = multiprocessing.Process(target=run_model, args=(lambda1, tau, X, device))
        processes.append(p)
        p.start()

# Wait for all processes to complete
for process in processes:
    process.join()

print("Grid search complete.")